In [35]:
from bs4 import BeautifulSoup
import requests
import pandas as pd 

url = 'https://www.zillow.com/ut/2_p/?searchQueryState=%7B%22isMapVisible%22%3Atrue%2C%22mapBounds%22%3A%7B%22north%22%3A42.001618%2C%22south%22%3A36.997903%2C%22east%22%3A-109.041058%2C%22west%22%3A-114.052998%7D%2C%22mapZoom%22%3A6%2C%22usersSearchTerm%22%3A%22Utah%22%2C%22regionSelection%22%3A%5B%7B%22regionId%22%3A55%2C%22regionType%22%3A2%7D%5D%2C%22filterState%22%3A%7B%22sort%22%3A%7B%22value%22%3A%22globalrelevanceex%22%7D%2C%22ah%22%3A%7B%22value%22%3Atrue%7D%7D%2C%22isListVisible%22%3Atrue%2C%22schoolId%22%3Anull%2C%22pagination%22%3A%7B%22currentPage%22%3A2%7D%7D'

headers = {
         "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.0.0 Safari/537.36"
}



    


res = requests.get(url , headers=headers)

soup =  BeautifulSoup(res.content, 'html.parser')

housing_list=[]
for index in soup.find_all('div',{'class':'StyledPropertyCardDataWrapper-c11n-8-105-0__sc-hfbvv9-0 jqlVkt property-card-data'}):

    price =index.find('span' , {'class':'PropertyCardWrapper__StyledPriceLine-srp-8-105-0__sc-16e8gqd-1 dHAgxu'}).text.strip() if index.find('span' , {'class':'PropertyCardWrapper__StyledPriceLine-srp-8-105-0__sc-16e8gqd-1 dHAgxu'}) else 'N/A'

    unordered_list = index.find('ul' , {'class':'StyledPropertyCardHomeDetailsList-c11n-8-105-0__sc-1j0som5-0 ldtVy'})
    list_items= unordered_list.find_all('li')
    bedroom = list_items[0].text.strip() if len(list_items[0].text.strip()) > 0 else 'N/A'
    bathroom =list_items[1].text.strip() if len(list_items[0].text.strip()) > 0 else 'N/A'
    area = list_items[2].text.strip() if len(list_items[0].text.strip()) > 0 else 'N/A'
    address = index.find('address').text.strip() if index.find('address') else 'N/A'

    housing_list.append({
        'Price':price,
        'Bedroom':bedroom,
        'Bathrooms':bathroom,
        'Area in sqft':area,
        'Address':address,
        
    })


df = pd.DataFrame(housing_list)
df.to_csv(r'E:\Data Analyst\Project\Scraped Dataset\Real_estate.csv' , index=False , mode='a' , header=False)
print('Data  Scraped and saved to csv file')



Data  Scraped and saved to csv file
Data  Scraped and saved to csv file
Data  Scraped and saved to csv file
Data  Scraped and saved to csv file
Data  Scraped and saved to csv file


MissingSchema: Invalid URL '': No scheme supplied. Perhaps you meant https://?

In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
import requests


url='https://www.cimls.com/new+york/new+york-commercial-real-estate'

headers = {
         "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.0.0 Safari/537.36"
}



res = requests.get(url , headers=headers)

soup = BeautifulSoup(res.content , 'html.parser')
commercial_list=[]
while True:
    url=input("Enter the Url")
    if url.lower() == 'exit':
        break

    if url.isdigit():
        break

    else:
        for index in soup.find_all('div', {'class':'listing-info'}):
            address  = index.find('div' , {'class':'listing-title'}).text.strip() if index.find('div' , {'class':'listing-title'}) else 'N/A'
            bold_tag = index.find('b').get_text(separator=' ').strip() if index.find('b')  else 'N/A'

            split_data = bold_tag.split(' ', 1)
            
            if len(split_data) == 2:
                price, area = split_data
                price = price.replace('$', ' ').strip()
            else:
                price = 'N/A'
                area = 'N/A'
            
            commercial_list.append({
                'Price':price,
                'Bedroom':'N/A',
                'Bathrooms':'N/A',
                'Area in sqft':area,
                'Address':address,
                'Type':'Commercial'
            })

        df = pd.DataFrame(commercial_list)
        df.to_csv(r'E:\Data Analyst\Project\Scraped Dataset\Real_estate.csv', index=False , mode='a')
        print("Data Saved Successfully")
        df 

Data Saved Successfully
Data Saved Successfully
Data Saved Successfully
Data Saved Successfully
Data Saved Successfully
Data Saved Successfully
Data Saved Successfully
Data Saved Successfully
Data Saved Successfully
Data Saved Successfully
Data Saved Successfully
Data Saved Successfully
Data Saved Successfully
Data Saved Successfully
Data Saved Successfully
Data Saved Successfully


In [23]:
#cleaning the dataset 
import numpy as np
import pandas as pd 
df = pd.read_csv('E:\Data Analyst\Project\Scraped Dataset\Real_estate.csv')
pd.set_option('display.max_rows' , None)

#df.drop('Unnamed:6' , axis=0)
df['Price'] = df['Price'].str.replace(r'\D' , '' , regex=True)
df['Price'] = df['Price'].str.strip()
df = df.dropna(subset=['Price'])
df = df[~df[df.columns].apply(lambda x: x.str.contains('Address', na=False)).any(axis=1)]

#we know that in regular expression \D represents all Values other than numbers
df[['Bedroom' , 'Bathrooms']] = df[['Bedroom', 'Bathrooms']].apply(lambda x: x.str.replace(r'\D' , '' , regex=True) )
df['Area in sqft'] = df['Area in sqft'].str.replace(r'Sq\. Ft\. on \d+(\.\d+)? acres?' , '' , regex=True).str.replace('sqft', '' , regex=True).str.replace('Sq\. Ft\.', '', regex=True)

df = df.dropna(subset=['Price'])
df =df[~df['Area in sqft'].str.contains(r'[a-zA-Z]' ,regex=True , na=True)]
df['Area in sqft'] = df['Area in sqft'].replace('',  np.nan)

df = df.dropna(subset='Area in sqft' ) # df.dropna(subset='Area in sqft' , inplace=True)

df['Pincode'] = df['Address'].astype(str).str.extract(r'(\d{5})(?!.*\d{5})') 


df.to_csv(r'E:\Data Analyst\Project\Scraped Dataset\Cleaned_Real_estate.csv')
print("Data saved successfully ")

Data saved successfully 


In [25]:
df=pd.read_csv(r'E:\Data Analyst\Project\Scraped Dataset\Cleaned_Real_estate.csv')
avg_price_by_location = df.groupby('Address')['Price'].mean().reset_index()
print(avg_price_by_location)

                                                Address        Price
0                    1 Montana Wild Dr, Libby, MT 59923     149900.0
1                    10 Ivanhoe Ct, Kissimmee, FL 34746      70000.0
2                          100 Elm St, Phoenix, Arizona    5400000.0
3              100 Harvest Hill Road, Windham, ME 04062     550000.0
4                            100 Oak St, Miami, Florida    6000000.0
5                       100 Union St, Weirton, WV 26062     239900.0
6               1000 Rotherwood Dr, Kingsport, TN 37660     674900.0
7                            101 Oak St, Columbus, Ohio    5500000.0
8                        101 Radecke Rd, Krum, TX 76249     330000.0
9        10113 N 93rd Ave, Mackenzie County, AB T0H 2H0     435000.0
10             1012 SW 95th St, Oklahoma City, OK 73139     225000.0
11                1013 Southwest Dr, Cheyenne, WY 82007     415000.0
12                     1015 Serena Dr, Dallas, TX 75253     285000.0
13                     102 Elm St,